## Prepare the dataset

### Create a data generator node
<img src="images/xgboost/create_node.gif" align="center">

### Explore and visualize the data
<img src="images/xgboost/visualize_data.gif" align="center">

### Add categorical variable
<img src="images/xgboost/categorical_variable.gif" align="center">

### Save the graph and create a composite node
<img src="images/xgboost/create_composite_node.gif" align="center">

## Preprocess the data

### Split the dataset into train and test
<img src="images/xgboost/split_the_dataset.gif" align="center">

### Normalize the features
<img src="images/xgboost/normalize.gif" align="center">

### Linear embedding
<img src="images/xgboost/linear_projection.gif" align="center">

### gQuant evaluates a subgraph, no wasted computation
<img src="images/xgboost/sub_graph_evaluation.gif" align="center">


## Machine Learning
### Train a XGBoost Model and run inference
<img src="images/xgboost/train_and_infer.gif" align="center">

### visualize the training result feature importance

### Model deployment


## Real life example

### Create a composite node for the machine learning graph
### Make a customized node out of the composite node
### Use the stock data 